In [12]:
from dotenv import load_dotenv
load_dotenv()

from pprint import pprint

In [41]:
from langchain.agents import Tool
from langchain.utilities import DuckDuckGoSearchAPIWrapper

def create_sited_search(sites = [], debug = False):
    def sited_search(query):
        site_queries = list(map(
                lambda site_str: f'site:{site_str}',
                sites
                ))
        joined_site_query = ' OR '.join(site_queries)

        search_query = query + ' ' + joined_site_query

        if debug:
            pprint(f"Search: {search_query}")

        return DuckDuckGoSearchAPIWrapper().results(search_query, num_results=5)
    
    return sited_search

sites = [
    'https://jinjibu.jp/',
    'https://hrnote.jp/'
]

search_tool = Tool(
    name='Search',
    description=(
        "A wrapper around Duck Duck Go Search. "
        "Useful for when you need to answer questions about current events. "
        "Input should be a search query. Output is a JSON array of the query results"
    ),
    func=create_sited_search(sites=sites, debug=True),
)

In [36]:
search_tool.run('退職')

'Search: 退職 site:https://jinjibu.jp/ OR site:https://hrnote.jp/'


/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/duckduckgo_search/compat.py:22: UserWarning: parameter time is deprecated, use parameter timelimit
  warnings.warn("parameter time is deprecated, use parameter timelimit")
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-package

[{'snippet': '従業員が退職する際は、健康保険・厚生年金保険や雇用保険を中心としたさまざまな手続きが発生します。これらの手続きにはそれぞれ期限が ...',
  'title': '従業員が退職するときの手続きと注意点 - 日本の人事部',
  'link': 'https://jinjibu.jp/qa/sum/retirement/'},
 {'snippet': '「静かな退職」とは、組織に在籍しながらも契約通りの仕事だけを淡々と行い、退職したかのように精神的な余裕を持って働くこと。米国を中心 ...',
  'title': '静かな退職（Quiet Quitting）とは――意味や日本での現状、解決のヒントを解説 - 『日本の人事部』',
  'link': 'https://jinjibu.jp/keyword/detl/1556/'},
 {'snippet': '退職に関する規定はさまざまな法律で定められているので、定義とともに違いを把握しておかなければ人事担当者の業務の妨げとなる可能性があります。本記事では特に労働基準法における退職やそれにともなう手続きについて確認し、速やかに業務を進められるようにしましょう。',
  'title': '労働基準法における退職の定義は？手続き方法や注意点を紹介 ｜Hr Note',
  'link': 'https://hrnote.jp/contents/roumu-rodokijunho-taishoku-20230116/'},
 {'snippet': '社員が退職するときには社会保険の手続きが必要となります。健康保険や厚生年金保険、雇用保険など、種類ごとに適切な対処をしたいものです。また、社員が退職する際に発行する書類の種類についても把握しておきましょう。',
  'title': '社員が退職したときの社会保険手続きについて徹底解説 ｜Hr Note',
  'link': 'https://hrnote.jp/contents/roumu-shakaihoken-taisyoku-20220901/'},
 {'snippet': '退職届をいつ提出するかは雇用契約の内容によって異なります。正社員のように雇用期間に定めがない場合は、遅くとも退職日の14日前までに退職を申し入れることが民法

In [44]:
from langchain.document_loaders import WebBaseLoader

def site_look_up(url):
    loader = WebBaseLoader(url)
    doc = loader.load()
    return doc

lookup_tool = Tool(
    name='Lookup',
    description=(
        "A wrapper Web Base Loader "
        "Useful for when you need to lookup "
        "Input should be a site URL Link. Output is a parsed String"
    ),
    func=site_look_up,
)

In [38]:
lookup_tool.run('https://jinjibu.jp/qa/sum/retirement/')

[Document(page_content='\n\n\n\n従業員が退職するときの手続きと注意点 - 『日本の人事部』\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n『日本の人事部』 - HRで会社を伸ばす\n\n\n\n 人事正会員数286162人\n\n\n\n\n\n『日本の人事部』とは\n\n\n\n人事カレンダー\n\n\n\nクリップ一覧\n\n\n\nログイン\n\n会員登録（無料）\n\n\n\n\n全て\nサービス\nセミナー\n資料・書式文例\n人事のQ&A\n記事・ニュース\n人事辞典\n\n\n\n\n\n×\n\n\n注目ワード\n\n採用人的資本リスキリング人事評価エンゲージメントハラスメント心理的安全性ウェルビーイング人材ポートフォリオHRBP \n\n\n閲覧トップコンテンツ\n\n\n\n\n\n\n\n記事\n\n\r\n                                    佐々木秀綱さん：\n権力は人をどう変えるのか？ リーダー育成や組織運営のために知っておきたい「社会的勢力感」の仕組み                                \n\n\n\n\n\n\n\n\n\n\n記事\n\n\r\n                                    テルモ株式会社：\n「20代でも課長になれる」大胆な公募制を導入\n若手の「能力」と「野心」を引き出す、テルモの若手登用・育成術とは                                \n\n\n\n\n\n\n\n\n\n\nニュース\n\n\r\n                                    2023年新入社員のキャリア意識調査                                \n\n\n\n\n\n\n\n\n 日本最大のHRイベント\r\n\t\t\t\t\t\t\tＨＲカンファレンス\r\n\t\t\t\t\t\t\n\n\n\n全国の人事が選ぶ\r\n\

In [49]:
from langchain import OpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [search_tool, lookup_tool]

llm = OpenAI(temperature=0, model_name="gpt-4")
react = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/norifusa.tachiyanagi/Library/Caches/pypoetry/virtualenvs/jinjiroumubot-0jSYAYbv-py3.11/lib/python3.11/site-packages/langchain/llms/openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [50]:
question = "I am a human resources manager working for a Japanese company. I would like to '退職' and would like to know exactly how to do so."
react.run(question)



> Entering new AgentExecutor chain...
I need to find out the process of resigning from a Japanese company.
Action: Search
Action Input: how to resign from a Japanese company('Search: how to resign from a Japanese company site:https://jinjibu.jp/ OR '
 'site:https://hrnote.jp/')

Observation: [{'snippet': '雇用保険被保険者資格喪失届は速やかな提出を. 1. 雇用保険被保険者資格喪失届の書き方を項目ごとに解説. 雇用保険被保険者資格喪失届は、被保険者の退職だけでなく、被保険者が死亡した場合や被保険者が会社の役員になった場合などにも必要となります ...', 'title': '雇用保険被保険者資格喪失届の書き方を項目ごとに詳しく解説 ｜Hr Note', 'link': 'https://hrnote.jp/contents/roumu-koyouhokenshikakusoushitsutodoke-kakikata-20220829/'}, {'snippet': '退職届をいつ提出するかは雇用契約の内容によって異なります。正社員のように雇用期間に定めがない場合は、遅くとも退職日の14日前までに退職を申し入れることが民法で定められています。引継ぎや後任の採用などを考慮して、退職を希望する従業員と相談して合意のうえで退職日を設定 ...', 'title': '退職届は何日前に必要？労働基準法による規定をわかりやすく紹介', 'link': 'https://hrnote.jp/contents/roumu-rodokijunho-taishoku-nannichimae-20230126/'}, {'snippet': '総務担当者は退職手続きの流れを理解してスムーズに進めよう. 1. 総務担当者が行うべき退職手続き. 従業員が退職する際に、総務担当者が行う手続きは多岐にわたります。. 退職日までにするべきことや、その後に行う手続きについて確認しましょう ..

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 10128 tokens. Please reduce the length of the messages.

In [18]:
tools[0].run('David Chanoff')

'David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.'